<h2>Regression Models with Keras. Part D</h2>
<h3>Objective for this Notebook:</h3>    
Repeat Regression Models with Keras Part B but use a neural network with the following instead:  

- Three hidden layers, each of 10 nodes and ReLU activation function.  

How does the mean of the mean squared errors compare to that from Step B?  
<h3>Concrete Data:</h3>    
The data can be found here: 
https://cocl.us/concrete_data
   
 
       


## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item41">Prepare the data: Download, Clean and Split the Dataset     
2. <a href="#item42">Import Keras</a>  
3. <a href="#item43">Build a Neural Network</a>  
4. <a href="#item44">Train and Test the Network</a>  
5. <a href="#item45">Evaluate the model</a>      

</font>
</div>


<a id="item41"></a>

## 1. Prepare the data: Download, Check and Split the Dataset

#### 1.1 Download the data 

Import the <em>pandas</em> and the Numpy libraries.


In [1]:
# Uncomment the following if running on desktop:
#!pip install numpy==1.21.4
#!pip install pandas==1.3.4
#!pip install keras==2.1.6

In [2]:
import pandas as pd
import numpy as np

Download the data and read it into a <em>pandas</em> dataframe.

In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


The dataset is about the compressive strength of different samples of concrete based on the volumes of the different ingredients that were used to make them. Ingredients include:

1. Cement
2. Blast Furnace Slag
3. Fly Ash
4. Water
5. Superplasticizer
6. Coarse Aggregate
7. Fine Aggregate


#### 1.2 Check the data 

Check how many data points we have.

In [4]:
concrete_data.shape

(1030, 9)

Check the dataset for any missing values.

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

#### 1.3 Split the data into predictors and target


In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

Sanity check of the predictors and the target dataframes.


In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

Normalize the data by substracting the mean and dividing by the standard deviation.


In [10]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Save the number of predictors to *n_cols* since we will need this number when building our network.


In [11]:
n_cols = predictors_norm.shape[1] # number of predictors

<a id="item42"></a>

## 2. Import Keras 
Import Keras and the packages from the Keras library that we will need to build our regression model.

In [12]:
import keras

Using TensorFlow backend.
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jupyterlab/con

In [13]:
from keras.models import Sequential
from keras.layers import Dense

<a id="item43"></a>

## 3. Build a Neural Network

Define a function that defines our regression model for us so that we can conveniently call it to create our model.

- Three hidden layers, each of 10 nodes and ReLU activation function

- Use the adam optimizer and the mean squared error as the loss function.


In [14]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Call the function to create our model.


In [15]:
# build the model
model = regression_model()

<a id="item44"></a>

## 4. Train and Test the Network


4.1. Randomly split the data into a training and test sets by holding 30% of the data for testing.  
4.2. Train the model on the training data using 50 epochs.

In [16]:
# fit the model
model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=2)



Train on 721 samples, validate on 309 samples
Epoch 1/50


2024-01-06 16:28:54.165917: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
2024-01-06 16:28:54.171943: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2394295000 Hz
2024-01-06 16:28:54.172989: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e562cf4990 executing computations on platform Host. Devices:
2024-01-06 16:28:54.173058: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2024-01-06 16:28:54.299110: W tensorflow/compiler/jit/mark_for_compilation_pass.cc:1412] (One-time warning): Not using XLA:CPU for cluster because envvar TF_XLA_FLAGS=--tf_xla_cpu_global_jit was not set.  If you want XLA:CPU, either set that envvar, or use experimental_jit_scope to enable XLA:CPU.  To confirm that XLA is active, pass --vmodule=xla_compilation_cache=1 (as a proper command-line fl

 - 0s - loss: 1784.3492 - val_loss: 1279.1835
Epoch 2/50
 - 0s - loss: 1746.4478 - val_loss: 1256.2749
Epoch 3/50
 - 0s - loss: 1720.2166 - val_loss: 1239.3579
Epoch 4/50
 - 0s - loss: 1700.7828 - val_loss: 1225.5559
Epoch 5/50
 - 0s - loss: 1685.4183 - val_loss: 1213.6706
Epoch 6/50
 - 0s - loss: 1670.8545 - val_loss: 1201.0454
Epoch 7/50
 - 0s - loss: 1652.9805 - val_loss: 1184.5025
Epoch 8/50
 - 0s - loss: 1627.2745 - val_loss: 1159.7904
Epoch 9/50
 - 0s - loss: 1587.3884 - val_loss: 1122.3247
Epoch 10/50
 - 0s - loss: 1527.3191 - val_loss: 1067.0070
Epoch 11/50
 - 0s - loss: 1436.1414 - val_loss: 986.6263
Epoch 12/50
 - 0s - loss: 1302.5613 - val_loss: 872.4970
Epoch 13/50
 - 0s - loss: 1111.4545 - val_loss: 733.2163
Epoch 14/50
 - 0s - loss: 877.1001 - val_loss: 576.0727
Epoch 15/50
 - 0s - loss: 624.6696 - val_loss: 429.6554
Epoch 16/50
 - 0s - loss: 412.2725 - val_loss: 313.6042
Epoch 17/50
 - 0s - loss: 283.6657 - val_loss: 245.7553
Epoch 18/50
 - 0s - loss: 232.1692 - val_loss

<a id="item45"></a>

## 5. Evaluate the model

5.1. Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength.    

In [17]:
## evaluate the model
from sklearn.metrics import mean_squared_error

# y_pred are the predictions of the model with the test data 
y_pred = model.predict(predictors_norm)

# Calculate Mean Squared Error (MSE) 
mse = mean_squared_error(target, y_pred)

print(f'Mean Squared Error on Test Data: {mse}')

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/sklearn/utils/validation.py:37: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  LARGE_SPARSE_SUPPORTED = LooseVersion(scipy_version) >= '0.14.0'


Mean Squared Error on Test Data: 135.70408598125027


5.2. Repeat steps 4.1, 4.2 and 5.1, 50 times, i.e., create a list of 50 mean squared errors.

In [18]:
num_repeats = 50
mse_list = []
for _ in range(num_repeats):
    
    model.fit(predictors_norm, target, validation_split=0.3, epochs=50, verbose=0)
    y_pred = model.predict(predictors_norm)
    mse = mean_squared_error(target, y_pred)
    # Add MSE to the list
    mse_list.append(mse)

print("List of Mean Squared Errors:")
print(mse_list)

List of Mean Squared Errors:
[114.82478480420694, 97.26617336028767, 82.02441849641954, 77.9548083860247, 79.28672378236898, 76.52302323338328, 73.45981613319812, 70.62971611445359, 71.56188655311621, 67.51179601592524, 65.75522421027297, 64.98988137827742, 63.23860626472458, 62.798916046844276, 60.86474292497149, 62.44370084944205, 61.780152000584394, 60.655695160923955, 60.20221634432932, 58.412059719242436, 59.578702343303206, 54.38354999645064, 52.758065748571596, 51.94829533316982, 52.06432321586791, 50.653670092245086, 51.70192655436964, 48.75295760063603, 49.24184680211494, 46.46281173420354, 47.25665303828428, 47.964917295448124, 45.69797583594313, 43.69687999745047, 44.71067089100065, 44.74899826970014, 44.71239493709966, 47.65108487787156, 43.991994459721205, 42.25316687409018, 43.490008329424356, 43.66489384042932, 43.403157948665395, 43.272179105361865, 42.00054791928517, 43.963552427858545, 42.80167707326734, 42.669005502704934, 41.52770459584703, 44.5607899926921]


5.3. Report the mean and the standard deviation of the mean squared errors.

In [19]:
mean_mse = np.mean(mse_list)
std_mse = np.std(mse_list)

print(f"Mean of Mean Squared Errors: {mean_mse}")
print(f"Standard Deviation of Mean Squared Errors: {std_mse}")

Mean of Mean Squared Errors: 56.67537488824149
Standard Deviation of Mean Squared Errors: 15.306463375672434


Mean of Mean Squared Errors: 56.67537488824149
Standard Deviation of Mean Squared Errors: 15.306463375672434
With an increased number of hidden layers and the same 50 epochs as part B, shows a lower mean MSE (56.68) compared to part B (68.37), suggesting that adding more hidden layers might contribute to improved performance. Additionally, the standard deviation in Case D is lower, indicating more consistent results